In [ ]:
!pip install tqdm
!pip install dwave-neal
!pip install networkx
!pip install pandas

import networkx as nx
import numpy as np
import random
from collections import defaultdict, Counter
from tqdm import tqdm
import neal
import pandas as pd
from dimod import BinaryQuadraticModel

In [ ]:
USE_QUBO_SOLVER = True
TOTAL_ROUNDS = 100
NUM_READS = 100
LOCAL_VIEW_RADIUS = 2
FLOORS = 5
X_ROOMS = 12
Y_ROOMS = 12
NUM_AGENTS = 100
CURING_BUDGET = int(NUM_AGENTS * 0.2)
BLOCK_FRACTION = 0.03
PROB_ATTACK_ROUND = 0.85
SPOOF_FRAC_RANGE = (0.05, 0.2)
SPOOF_SEVERITY = 0.1

In [ ]:
def build_synthetic_graph(floors=FLOORS, x_rooms=X_ROOMS, y_rooms=Y_ROOMS, block_fraction=BLOCK_FRACTION):
    G = nx.Graph()
    for f in range(floors):
        for x in range(x_rooms):
            for y in range(y_rooms):
                node = f"R{f}_{x}_{y}"
                G.add_node(node, floor=f, x=x, y=y,
                           blocked=False,
                           urgency=random.uniform(0.5, 1.0),
                           structural_integrity=1.0,
                           is_bait=False,
                           is_exit=False)
    for f in range(floors):
        for x in range(x_rooms):
            for y in range(y_rooms):
                current = f"R{f}_{x}_{y}"
                if x < x_rooms - 1:
                    G.add_edge(current, f"R{f}_{x+1}_{y}", status="open")
                if y < y_rooms - 1:
                    G.add_edge(current, f"R{f}_{x}_{y+1}", status="open")
                if f < floors - 1:
                    G.add_edge(current, f"R{f+1}_{x}_{y}", status="open")
    num_blocked = int(block_fraction * G.number_of_nodes())
    blocked_nodes = random.sample(list(G.nodes()), min(num_blocked, G.number_of_nodes()))
    for node in blocked_nodes:
        G.nodes[node]["blocked"] = True
    G.nodes[f"R0_0_0"]["is_exit"] = True
    G.nodes[f"R0_0_{y_rooms-1}"]["is_exit"] = True
    G.nodes[f"R0_{x_rooms-1}_0"]["is_exit"] = True
    G.nodes[f"R0_{x_rooms-1}_{y_rooms-1}"]["is_exit"] = True
    bait_count = int(0.05 * G.number_of_nodes())
    bait_nodes = random.sample(list(G.nodes()), min(bait_count, G.number_of_nodes()))
    for node in bait_nodes:
        G.nodes[node]["is_bait"] = True
    return G

In [ ]:
def generate_synthetic_tasks(graph, num_tasks=50, room_pool=None):
    if room_pool is None:
        room_pool = [n for n in graph.nodes if not graph.nodes[n].get("blocked", False) and graph.nodes[n].get("type") != "exit"]

    if len(room_pool) < num_tasks:
        print(f"Only {len(room_pool)} taskable rooms available — reducing task count.")

    selected_rooms = random.sample(room_pool, min(num_tasks, len(room_pool)))
    tasks = {}

    for i, task_room in enumerate(selected_rooms):
        tasks[f"T{i}"] = {
            "room": task_room,
            "urgency": graph.nodes[task_room].get("urgency", random.uniform(0.5, 1.0)),
            "skills_required": [random.randint(0, 1) for _ in range(3)],
            "is_team_task": random.random() < 0.3,
            "is_bait": graph.nodes[task_room].get("is_bait", False)
        }

    return tasks


def assign_exit_rooms(graph, exit_frac=0.1):
    rooms = list(graph.nodes)
    num_exits = max(1, int(exit_frac * len(rooms)))
    exit_rooms = random.sample(rooms, num_exits)
    for r in exit_rooms:
        graph.nodes[r]['is_exit'] = True
    return exit_rooms


def init_synthetic_robots(graph, tasks, num_agents):
    robots = {}
    valid_rooms = list(graph.nodes)

    for agent_id in range(num_agents):
        assigned_room = random.choice(valid_rooms)
        skill_count = min(10, len(tasks)) if tasks else 1
        skills = [round(random.uniform(0.5, 1.0), 2) for _ in range(skill_count)]

        robots[agent_id] = {
            "position": assigned_room,
            "assigned_room": assigned_room,
            "last_room": None,
            "trust": round(random.uniform(0.5, 0.95), 2),
            "trust_history": [],
            "compromised": False,
            "comms_down": False,
            "must_exit": False,
            "pending_reward": 0.0,
            "success_count": 0,
            "fail_count": 0,
            "idle_rounds": 0,
            "spoofed_room": None,
            "skills": skills
        }

    return robots


def assign_chained_precedence(tasks, chain_length=3, num_chains=5):
    available = list(tasks.keys())
    random.shuffle(available)
    pairs = set()

    for _ in range(min(num_chains, len(available) // chain_length)):
        chain = available[:chain_length]
        available = available[chain_length:]

        for i in range(1, len(chain)):
            tasks[chain[i]]['must_follow'] = chain[i - 1]
            pairs.add((chain[i - 1], chain[i]))

    return pairs


def define_precedence_map(precedence_pairs):
    mapping = {}
    for a, b in precedence_pairs:
        mapping.setdefault(b, []).append(a)
    return mapping

def simulate_outcome_with_path_check(graph, robot, task, exit_rooms):
    temp_graph = graph.copy()

    blocked_nodes = [n for n, d in graph.nodes(data=True) if d.get("blocked", False)]
    temp_graph.remove_nodes_from(blocked_nodes)
    blocked_edges = [(u, v) for u, v, d in graph.edges(data=True) if d.get("status") == "blocked"]
    temp_graph.remove_edges_from(blocked_edges)

    start = robot.get("position", task)
    end = task

    if start not in temp_graph:
        print(f"Robot start node {start} not in graph")
        return False
    if end not in temp_graph:
        print(f" Task node {end} not in graph")
        return False

    try:
        nx.shortest_path(temp_graph, source=start, target=end)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        print(f" No path from {start} to task {end}")
        return False

    for exit_room in exit_rooms:
        if exit_room not in temp_graph:
            continue
        try:
            nx.shortest_path(temp_graph, source=end, target=exit_room)
            return True
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue

    print(f" No exit reachable from task {end}")
    return False


def reset_spoofed_rooms(robots):
    for r in robots.values():
        r['spoofed_room'] = r['assigned_room']

def apply_environmental_noise(robots, noise_level=0.05):
    for r in robots.values():
        if not r['compromised']:
            factor = random.uniform(1 - noise_level, 1 + noise_level)
            r['trust'] = min(max(r['trust'] * factor, 0.0), 1.0)

def cure_spoofed_robots(robots, graph, budget_frac=0.2, recovery_boost=0.5):
    compromised_agents = [r for r in robots.values() if r['compromised']]
    num_to_cure = int(len(robots) * budget_frac)
    agents_to_cure = random.sample(compromised_agents, min(num_to_cure, len(compromised_agents)))
    valid_rooms = list(graph.nodes)
    random.sample(compromised_agents, min(num_to_cure, len(compromised_agents)))
    for r in agents_to_cure:
        r['compromised'] = False
        r['trust'] = min(r['trust'] + recovery_boost, 1.0)

def simulate_comms_failures(robots, drop_prob=0.2):
    for r in robots.values():
        r['comms_down'] = (random.random() < drop_prob)

def decay_task_urgency(graph, decay_rate=0.2):
    for n, data in graph.nodes(data=True):
        if data.get('type') == 'task':
            data['urgency'] = max(data.get('urgency', 0.0) - decay_rate, 0.0)

def update_urgency(graph, robots, tasks):
    treated_tasks = [r["assigned_room"] for r in robots.values() if r.get("success", False)]
    nearby_counts = Counter(treated_tasks)

    for t in tasks:
        room = tasks[t].get("room")
        if room is None or room not in graph.nodes:
            continue

        count = nearby_counts[room]
        boost = 0.02 + 0.1 * count
        graph.nodes[room]["urgency"] = min(1.0, graph.nodes[room].get("urgency", 0.0) + boost)


def update_dynamic_blockages(graph, block_prob=0.03):
    for (u, v, data) in list(graph.edges(data=True)):
        if data.get('status') != 'blocked' and random.random() < block_prob:
            graph.edges[u, v]['status'] = 'blocked'

def precompute_static_costs(robots, tasks, graph):
    static = {}
    quadratic = {}

    for r_id, r in robots.items():
        for t_id, t in tasks.items():
            if r['compromised']:
                continue

            trust = r['trust']
            integrity = t.get("structural_integrity", 1.0)
            base_cost = 1.0 - trust * integrity

            static[(r_id, t_id)] = round(base_cost, 4)

    for t_id, t in tasks.items():
        if t.get("team_size", 1) > 1:
            involved_agents = [r_id for r_id in robots if not robots[r_id]['compromised']]
            for a1 in involved_agents:
                for a2 in involved_agents:
                    if a1 != a2:
                        quadratic[((a1, t_id), (a2, t_id))] = -0.25

    return static, quadratic

def decay_urgency(graph):
    for node in graph.nodes:
        curr = graph.nodes[node].get('urgency', 0.5)
        if graph.nodes[node].get('is_exit'):
            continue
        decay = random.uniform(0.01, 0.03)
        graph.nodes[node]['urgency'] = round(max(0.1, curr - decay), 2)

def enforce_exit_penalties(assignments, graph, exit_rooms):
    violations = 0
    for agent_id, task_id in assignments.items():
        if isinstance(task_id, list):
            task_id = task_id[0]
        if task_id not in graph.nodes:
            continue
        neighbors = list(graph.neighbors(task_id)) + [task_id]
        if not any(graph.nodes[n].get("is_exit") for n in neighbors):
            violations += 1
    return violations
def assign_bait_and_team_tasks(graph, tasks, bait_frac=0.1, team_frac=0.2):
    task_ids = list(tasks.keys())
    num_bait = int(bait_frac * len(task_ids))
    num_team = int(team_frac * len(task_ids))

    bait_tasks = random.sample(task_ids, num_bait)
    valid_rooms = list(graph.nodes)
    random.sample(task_ids, num_bait)
    for tid in bait_tasks:
        tasks[tid]['is_bait'] = True
        tasks[tid]['structural_integrity'] = 0.1

    team_tasks = random.sample([tid for tid in task_ids if tid not in bait_tasks], num_team)
    valid_rooms = list(graph.nodes)
    random.sample([tid for tid in task_ids if tid not in bait_tasks], num_team)
    for tid in team_tasks:
        tasks[tid]['team_size'] = 2

    for tid in task_ids:
        tasks[tid].setdefault('structural_integrity', 1.0)
        tasks[tid].setdefault('team_size', 1)

def update_dynamic_costs_np(robots, tasks, graph, static_terms, dist_matrix,
                            dist_penalty=1.0, comp_penalty=1.0, local_radius=3,
                            duplicate_soft_penalty=0.5):

    cost_dict = {}

    for r_id, robot in robots.items():
        if robot.get("compromised"):
            continue
        skill_list = robot.get("skills", [1.0])
        assigned = set(robot.get("assigned_tasks", []))

        for t in tasks:
            room = tasks[t]['room']
            start_pos = robot["position"]
            dist = dist_matrix.get((start_pos, room), random.randint(5, 20))

            urgency = graph.nodes[room].get("urgency", 0.5)

            if dist > local_radius and urgency < 0.2:
                continue

            key = (r_id, t)
            trust = robot.get("trust", 1.0)
            spoofed = robot.get("spoofed", False)
            skill = skill_list[0] if skill_list else 1.0

            static_val = static_terms.get(key, 5.0)
            static_val = min(static_val, 50.0)

            cost = static_val
            cost += dist_penalty * dist
            cost += comp_penalty * int(spoofed)
            cost -= 15 * trust * urgency * skill
            cost += random.uniform(-0.5, 0.5)

            cost_dict[key] = cost

            if random.random() < 0.01:
                print(f"Agent {r_id} → Task {t} | Trust={trust:.2f}, Urg={urgency:.2f}, "
                      f"Skill={skill:.2f}, Cost={cost:.2f}, Dist={dist}")

    return cost_dict

def greedy_assignment_synthetic(robots, tasks, graph, precedence_map, dist_matrix, exit_rooms, use_synergy=True, local_radius=4):
    task_to_agents = defaultdict(list)
    assigned_tasks = set()

    for agent_id, robot in robots.items():
        current_pos = robot["position"]
        viewable_tasks = apply_local_view(tasks, current_pos, graph, radius=local_radius)
        best_task = None
        best_score = float('-inf')
        fallback_task = None
        fallback_score = float('-inf')

        for task in viewable_tasks:
            if task in assigned_tasks:
                continue

            urgency = graph.nodes[task].get("urgency", 0)
            if urgency > fallback_score:
                fallback_score = urgency
                fallback_task = task

            if task in precedence_map:
                unmet = [p for p in precedence_map[task] if not any(r.get("assigned_room") == p for r in robots.values())]
                if unmet:
                    continue

            skill = robot["skills"][tasks.index(task) % len(robot["skills"])]
            trust = robot.get("trust", 1.0)
            distance = dist_matrix.get((current_pos, task), 20)
            score = urgency + trust * skill - 0.2 * distance

            if score > best_score:
                best_score = score
                best_task = task

        final_task = best_task or fallback_task
        if final_task:
            robot["assigned_room"] = final_task
            task_to_agents[final_task].append(agent_id)
            assigned_tasks.add(final_task)
            if not best_task:
                print(f"Agent {agent_id} using fallback task {final_task}")
        else:
            print(f" Agent {agent_id} found no task from {current_pos}")

    return task_to_agents



def build_or_update_bqm(bqm, cost_dict, quad_terms):
    if bqm is None:
        bqm = BinaryQuadraticModel('BINARY')
        for var in cost_dict:
            bqm.add_variable(var, 0.0)
        for (var1, var2), penalty in quad_terms.items():
            if penalty != 0.0:
                bqm.add_interaction(var1, var2, penalty)
    for var, cost in cost_dict.items():
        bqm.set_linear(var, cost)
    return bqm

def apply_qubo_assignments(result, robots, tasks, precedence_map, graph):
    assignments = {}
    current_tasks = set(tasks.keys())

    for (agent, task), val in result.items():
        if val != 1:
            continue

        deps = precedence_map.get(task, [])
        if not all(dep in current_tasks for dep in deps):
            continue

        room = tasks[task]['room']
        required_team = graph.nodes[room].get('required_team_size', 1)

        if required_team > 1:
            count = len([a for a, t in assignments.items() if t == task])
            if count + 1 < required_team:
                continue

        assignments[agent] = task

    return assignments

def spoofing_attack_target_high_trust(graph, robots, tasks, dist_matrix, spoof_frac_range=SPOOF_FRAC_RANGE, severity=SPOOF_SEVERITY):
    num_agents = len(robots)
    spoof_frac = random.uniform(*spoof_frac_range)
    num_to_spoof = int(spoof_frac * num_agents)
    if num_to_spoof <= 0:
        return

    task_ids = list(tasks.keys())

    candidates = sorted(robots.items(), key=lambda x: x[1]["trust"], reverse=True)
    top_half = candidates[:max(1, num_agents // 2)]
    selected = random.sample(top_half, min(num_to_spoof, len(top_half)))
    valid_rooms = list(graph.nodes)
    random.sample(top_half, min(num_to_spoof, len(top_half)))

    for rid, robot in selected:
        robot["compromised"] = True
        robot["spoofed_room"] = random.choice(task_ids) if task_ids else robot.get("assigned_room", None)
        robot["trust"] = max(0.1, robot["trust"] - severity)

    for rid, robot in robots.items():
        if robot["compromised"]:
            neighbors = [
                oid for oid, other in robots.items()
                if not other["compromised"]
                and dist_matrix.get((robot["assigned_room"], other["assigned_room"]), 999) <= LOCAL_VIEW_RADIUS
            ]
            if neighbors and random.random() < 0.5:
                target_id = random.choice(neighbors)
                target = robots[target_id]
                target["compromised"] = True
                target["spoofed_room"] = random.choice(task_ids) if task_ids else target.get("assigned_room", None)
                target["trust"] = max(0.1, target["trust"] - severity / 2)


In [ ]:
def run_metrics(robots, assignments, tasks, precedence_map, dist_matrix, graph, exit_rooms):
    total_agents = len(robots)
    assignment_counts = Counter(assignments.values())
    spoofed_agents = [r for r in robots.values() if r.get("compromised")]

    distances, urgency_vals, recovery_vals = [], [], []
    success_vals, reward_vals, trust_coord_vals = [], [], []
    duplicate_penalty = 0
    precedence_violations = 0
    bait_penalty = 0
    exit_violations = 0

    for agent_id, assigned_task in assignments.items():
        if agent_id not in robots:
            continue
        r = robots[agent_id]

        if assigned_task not in tasks:
            print(f"Task {assigned_task} not found in tasks.")
            continue

        room = tasks[assigned_task]['room']
        start_pos = r.get("position", room)
        dist = dist_matrix.get((start_pos, room), random.randint(5, 20))
        required_team = graph.nodes[room].get('required_team_size', 1)

        if required_team > 1 and assignment_counts[assigned_task] < required_team:
            print(f"Agent {agent_id} failed: not enough agents for team task {assigned_task}")
            r["fail_count"] += 1
            continue

        if not simulate_outcome_with_path_check(graph, r, room, exit_rooms):
            print(f"Agent {agent_id} failed: no valid path to task room {room}")
            r["fail_count"] += 1
            continue

        if r.get("last_room") == room:
            print(f"Agent {agent_id} failed: already visited room {room}")
            r["fail_count"] += 1
            continue

        r["position"] = room
        r["last_room"] = room
        r["success_count"] += 1

        if r.get("must_exit", False):
            if not graph.nodes[room].get("is_exit", False):
                print(f"Agent {agent_id} failed EXIT: {room} is not an exit")
                exit_violations += 1
                r["fail_count"] += 1
                trust_coord_vals.append(0)
                success_vals.append(0)
                distances.append(dist)
                urgency_vals.append(graph.nodes[room].get("urgency", 0.0))
                continue
            else:
                print(f"Agent {agent_id} exited via {room}")
                r["must_exit"] = False
                reward_vals.append(r.get("pending_reward", 0.0))
                r["pending_reward"] = 0.0
                trust_coord_vals.append(r["trust"])
                r["trust"] = min(1.0, r["trust"] + 0.01)
        else:
            r["must_exit"] = True
            urgency = graph.nodes[room].get("urgency", 0.0)
            r["pending_reward"] = urgency * r["trust"]

        urgency_vals.append(graph.nodes[room].get("urgency", 0.0))
        distances.append(dist)
        success_vals.append(1)

        attempts = r["success_count"] + r["fail_count"]
        recovery = r["success_count"] / attempts if attempts > 0 else 0.0
        recovery_vals.append(recovery)

        if assignment_counts[assigned_task] > 1:
            duplicate_penalty += 1

        if assigned_task in precedence_map:
            deps = precedence_map[assigned_task]
            completed_tasks = {rob["assigned_room"] for rob in robots.values()}
            if not all(dep in completed_tasks for dep in deps):
                precedence_violations += 1

        if graph.nodes[room].get("is_bait", False):
            bait_penalty += 1

    avg_trust = np.mean([r["trust"] for r in robots.values()])
    avg_urgency = np.mean(urgency_vals) if urgency_vals else 0.0
    avg_recovery = np.mean(recovery_vals) if recovery_vals else 0.0
    avg_coord_success = np.mean(success_vals) if success_vals else 0.0
    avg_spoofed = len(spoofed_agents) / total_agents if total_agents > 0 else 0.0
    avg_distance = np.mean(distances) if distances else 0.0
    avg_violations = ((exit_violations + precedence_violations) / len(assignments)) if assignments else 0.0
    avg_duplicates = (duplicate_penalty / len(assignments)) if assignments else 0.0
    coalition_success = np.mean([1 if count >= 2 else 0 for count in assignment_counts.values()]) if assignment_counts else 0.0
    trust_weighted_coord = np.mean(trust_coord_vals) if trust_coord_vals else 0.0
    avg_bait = bait_penalty
    avg_exit_violations = exit_violations
    reward_score = np.mean(reward_vals) if reward_vals else 0.0

    return {
        "avg_trust": round(avg_trust, 3),
        "trusturgency": round(avg_trust * avg_urgency, 3),
        "avg_spoofed": round(avg_spoofed, 3),
        "avg_recovery": round(avg_recovery, 3),
        "avg_coord_success": round(avg_coord_success, 3),
        "avg_violations": round(avg_violations, 3),
        "avg_distance": round(avg_distance, 2),
        "avg_duplicates": round(avg_duplicates, 3),
        "avg_bait": round(avg_bait, 3),
        "avg_exit_violations": round(avg_exit_violations, 3),
        "reward_score": round(reward_score, 3),
        "trust_weighted_coord": round(trust_weighted_coord, 3),
        "coalition_success": round(coalition_success, 3)
    }


In [ ]:
def run_synthetic_simulation(use_qubo=True, run_id="synth_run"):
    graph = build_synthetic_graph(FLOORS, X_ROOMS, Y_ROOMS, BLOCK_FRACTION)
    tasks = generate_synthetic_tasks(graph, num_tasks=50)
    all_pairs = dict(nx.all_pairs_shortest_path_length(graph))
    dist_matrix = {(src, tgt): d for src, dist_map in all_pairs.items() for tgt, d in dist_map.items()}

    robots = init_synthetic_robots(graph, tasks, NUM_AGENTS)
    exit_rooms = assign_exit_rooms(graph)
    precedence_pairs = assign_chained_precedence(tasks)
    precedence_map = define_precedence_map(precedence_pairs)
    sampler = neal.SimulatedAnnealingSampler()
    bqm = None

    trust_over_time = []
    trusturgency_over_time = []
    spoofed_over_time = []
    recovery_over_time = []
    coord_success_over_time = []
    violations_over_time = []
    distance_over_time = []
    bait_over_time = []
    exit_violations_over_time = []
    duplicates_over_time = []
    reward_score_over_time = []
    trust_weighted_over_time = []
    coalition_success_over_time = []

    summary_over_rounds = []
    initial_trust_avg = np.mean([r["trust"] for r in robots.values()])
    recent_task_rooms = []

    for round_num in tqdm(range(TOTAL_ROUNDS), desc=f"Running {'QUBO' if use_qubo else 'Greedy'} Simulation", leave=False):
        for robot in robots.values():
            robot['visited'] = set()

        reset_spoofed_rooms(robots)
        update_dynamic_blockages(graph)
        decay_task_urgency(graph)
        apply_environmental_noise(robots)
        cure_spoofed_robots(robots, graph, CURING_BUDGET / NUM_AGENTS)
        simulate_comms_failures(robots)

        if round_num % 3 == 0 or not tasks:
            available_rooms = [r for r in graph.nodes if r not in recent_task_rooms and graph.nodes[r].get("type") != "exit"]
            tasks = generate_synthetic_tasks(graph, num_tasks=50, room_pool=available_rooms)
            precedence_pairs = assign_chained_precedence(tasks)
            precedence_map = define_precedence_map(precedence_pairs)
            exit_rooms = assign_exit_rooms(graph)
            recent_task_rooms = [tasks[t]["room"] for t in tasks]

        if use_qubo:
            static_terms, quad_terms = precompute_static_costs(robots, tasks, graph)
            cost_dict = update_dynamic_costs_np(robots, tasks, graph, static_terms, dist_matrix)
            if not cost_dict:
                print(f"Round {round_num}: Skipping — empty QUBO cost dict.")
                continue
            bqm = build_or_update_bqm(bqm, cost_dict, quad_terms)
            result = sampler.sample(bqm, num_reads=NUM_READS).first.sample
            assignments = apply_qubo_assignments(result, robots, tasks, precedence_map, graph)
        else:
            assignments = hardened_greedy_assignment(robots, tasks, graph, precedence_map, dist_matrix, local_radius=LOCAL_VIEW_RADIUS)

        if not assignments:
            print(f"Round {round_num}: No assignments made.")
            continue

        for r_id, r in robots.items():
            if r_id not in assignments:
                r["idle_rounds"] += 1
                r["trust"] = max(0.0, r["trust"] - 0.002 * r["idle_rounds"])
            else:
                r["idle_rounds"] = 0
            if r.get("pending_reward", 0.0) > 0.02:
                r["trust"] = min(1.0, r["trust"] + 0.005)

        if random.random() < PROB_ATTACK_ROUND:
            spoofing_attack_target_high_trust(graph, robots, tasks, dist_matrix)

        update_urgency(graph, robots, tasks)

        metrics = run_metrics(robots, assignments, tasks, precedence_map, dist_matrix, graph, exit_rooms)

        summary_over_rounds.append(dict(
            round=round_num,
            avg_trust=metrics["avg_trust"],
            trusturgency=metrics["trusturgency"],
            avg_spoofed=metrics["avg_spoofed"],
            avg_recovery=metrics["avg_recovery"],
            avg_coord_success=metrics["avg_coord_success"],
            avg_violations=metrics["avg_violations"],
            avg_distance=metrics["avg_distance"],
            avg_duplicates=metrics["avg_duplicates"],
            avg_bait=metrics["avg_bait"],
            avg_exit_violations=metrics["avg_exit_violations"],
            reward_score=metrics["reward_score"],
            trust_weighted_coord=metrics["trust_weighted_coord"],
            coalition_success=metrics["coalition_success"]
        ))

        if round_num == 0:
            print(" First round keys:", list(summary_over_rounds[0].keys()))

        trust_over_time.append(metrics["avg_trust"])
        trusturgency_over_time.append(metrics["trusturgency"])
        spoofed_over_time.append(metrics["avg_spoofed"])
        recovery_over_time.append(metrics["avg_recovery"])
        coord_success_over_time.append(metrics["avg_coord_success"])
        violations_over_time.append(metrics["avg_violations"])
        distance_over_time.append(metrics["avg_distance"])
        bait_over_time.append(metrics["avg_bait"])
        exit_violations_over_time.append(metrics["avg_exit_violations"])
        duplicates_over_time.append(metrics["avg_duplicates"])
        reward_score_over_time.append(metrics["reward_score"])
        trust_weighted_over_time.append(metrics["trust_weighted_coord"])
        coalition_success_over_time.append(metrics["coalition_success"])

    final_trust = np.mean(trust_over_time) if trust_over_time else 0.0
    delta_trust = final_trust - initial_trust_avg
    avg_reward_score = np.mean(reward_score_over_time) if reward_score_over_time else 0.0
    avg_trust_weighted = np.mean(trust_weighted_over_time) if trust_weighted_over_time else 0.0
    trust_effort = avg_trust_weighted / (final_trust + 1e-6) if final_trust > 0 else 0.0
    delta_per_reward = delta_trust / (avg_reward_score + 1e-6) if avg_reward_score > 0 else 0.0

    summary = {
        "method": "QUBO" if use_qubo else "Greedy",
        "run_id": run_id,
        "final_trust": round(final_trust, 3),
        "initial_trust_avg": round(initial_trust_avg, 3),
        "delta_trust": round(delta_trust, 3),
        "delta_per_reward": round(delta_per_reward, 3),
        "trust_effort": round(trust_effort, 3),
        "trusturgency": round(np.mean(trusturgency_over_time), 3),
        "avg_spoofed": round(np.mean(spoofed_over_time), 3),
        "avg_recovery": round(np.mean(recovery_over_time), 3),
        "avg_coord_success": round(np.mean(coord_success_over_time), 3),
        "avg_violations": round(np.mean(violations_over_time), 3),
        "avg_distance": round(np.mean(distance_over_time), 2),
        "avg_bait": round(np.mean(bait_over_time), 3),
        "avg_exit_violations": round(np.mean(exit_violations_over_time), 3),
        "avg_duplicates": round(np.mean(duplicates_over_time), 3),
        "reward_score": round(avg_reward_score, 3),
        "trust_weighted_coord": round(avg_trust_weighted, 3),
        "coalition_success": round(np.mean(coalition_success_over_time), 3)
    }

    if summary_over_rounds:
        df = pd.DataFrame(summary_over_rounds)
        filepath = f"synthetic_rounds_{'qubo' if use_qubo else 'greedy'}_{run_id}.csv"
        df.to_csv(filepath, index=False)
    else:
        print("No rounds completed — skipping CSV save.")

    return summary

In [ ]:
run_id = "qubo_synth" if USE_QUBO_SOLVER else "greedy_synth"
result = run_synthetic_simulation(use_qubo=USE_QUBO_SOLVER, run_id=run_id)
filename = f"synthetic_summary_{'qubo' if USE_QUBO_SOLVER else 'greedy'}.csv"
pd.DataFrame([result]).to_csv(filename, index=False)
print(f"{filename} saved successfully!")